In [1]:
from modules.ir_1w1a import Nomal_conv2d
from modules.ir_1w1a import Nomal_linear
from modules import ir_1w1a
from vgg import VGG_SMALL_1W1A
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time
import math
import numpy
import os
import my_model

# Hyper parameters
momentum = 0.9
weight_decay = 1e-4
batch_size = 256


criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(net.parameters(), lr=lr)
#optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

best_acc = 0


# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load data

trainset = torchvision.datasets.CIFAR10(root='../../../data', train=True, download=True,
                                        transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='../../../data', train=False, download=True,
                                       transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')


def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def train(net, epoch=0):
    #criterion = nn.CrossEntropyLoss()
    #optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    #optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    print('\nEpoch: %d' % (epoch+1))
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        print('-', end='')
    print('\nTrain: Loss: {:.3f} | Acc: {:.3f}%% ({}/{})'.
          format(train_loss, 100. * correct / total, correct, total))
    return 100. * correct / total


def test(net):
    global best_acc
    criterion = nn.CrossEntropyLoss()
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        print('Test: Loss: {:.3f} | Acc: {:.3f}%% ({}/{})'.
              format(test_loss, 100. * correct / total, correct, total))
    return 100. * correct / total

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [5]:
model = my_model.VGG_SMALL_allnormal(q_bit=16).cuda()
lr = 0.007
epochs = 1000
criterion = nn.CrossEntropyLoss()
momentum = 0.9
weight_decay = 1e-4
optimizer = optim.SGD(model.parameters(), lr=lr,
                      momentum=momentum, weight_decay=weight_decay)

lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, epochs, eta_min=0, last_epoch=-1)
train(model)


Epoch: 1
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Train: Loss: 423.004 | Acc: 25.614%% (12807/50000)


25.614

In [6]:
test(model)

Test: Loss: 82.213 | Acc: 30.150%% (3015/10000)


30.15

In [74]:
# 将卷积层和bn层合并
def fuse(conv, bn):
    w = conv.weight
    mean = bn.running_mean
    var_sqrt = torch.sqrt(bn.running_var + bn.eps)

    beta = bn.weight
    gamma = bn.bias

    if conv.bias is not None:
        b = conv.bias
    else:
        b = mean.new_zeros(mean.shape)

    w = w * (beta / var_sqrt).reshape([conv.out_channels, 1, 1, 1])
    b = (b - mean)/var_sqrt * beta + gamma
    fused_conv = nn.Conv2d(conv.in_channels,
                         conv.out_channels,
                         conv.kernel_size,
                         conv.stride,
                         conv.padding,
                         bias=True)
    fused_conv.weight = nn.Parameter(w)
    fused_conv.bias = nn.Parameter(b)
    return fused_conv


class DummyModule(nn.Module):
    def __init__(self):
        super(DummyModule, self).__init__()

    def forward(self, x):
        # print("Dummy, Dummy.")
        return x
    
    
def fuse_module(m):
    children = list(m.named_children())
    c = None
    cn = None

    for name, child in children:
        if isinstance(child, nn.BatchNorm2d):
            bc = fuse(c, child)
            m._modules[cn] = bc
            m._modules[name] = DummyModule()
            c = None
        elif isinstance(child, nn.Conv2d):
            c = child
            cn = name
        else:
            fuse_module(child)

ttt=fuse(model.conv0,model.bn0).weight
print(ttt.mean())

tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)


In [51]:
from tqdm import tqdm
import struct
import os
import time
####################################################################################
## 将二进制的权重的list组合成uint8的list，便于写入文件
def bi_list_To_uint8_list(bi_list,n_1=1.0,n_0=-1.0):
    if type(bi_list)==numpy.ndarray:
        bi_list=bi_list.tolist()
    lenth=len(bi_list)

    # 对齐到字节
    if lenth%8!=0:
        for i in range(8-(lenth%8)):
            bi_list.append(n_0)
        lenth=lenth+8-lenth%8

    uint8_list=[]

    for i in range(lenth):
        if bi_list[i]==n_1:
            bi_list[i]=1
        elif bi_list[i]==n_0:
            bi_list[i]=0
        elif bi_list[i]==0.0:
            bi_list[i]=1
        else:
            print('error',bi_list[i])
            return

    for i in range(0,lenth,8):
        uint8_i=bi_list[i]*128+bi_list[i+1]*64+bi_list[i+2]*32+bi_list[i+3]*16+bi_list[i+4]*8+bi_list[i+5]*4+bi_list[i+6]*2+bi_list[i+7]
        uint8_i=int(uint8_i)
        uint8_list.append(uint8_i)
    return uint8_list

####################################################################################
model_path='./out/partnormal88.97/'
model = my_model.VGG_SMALL_1W1A_normal().cuda()

for i in tqdm(range(245,998)):

    ####################################################################################
    ### 保存两个版本的参数

    model.load_state_dict(torch.load(model_path+str(i)+'.ckpt'))
    first0=numpy.round(127*model.conv0.true_weight().detach().cpu().numpy().ravel()).astype(int)
    ## 1-5为二值化层
    first1=model.conv1.true_weight().detach().cpu().numpy().ravel()
    first2=model.conv2.true_weight().detach().cpu().numpy().ravel()
    first3=model.conv3.true_weight().detach().cpu().numpy().ravel()
    first4=model.conv4.true_weight().detach().cpu().numpy().ravel()
    first5=model.conv5.true_weight().detach().cpu().numpy().ravel()


    first_fc_weight=numpy.round(127*model.fc.true_weight()[0].detach().cpu().numpy().ravel()).astype(int)
    first_fc_bias=numpy.round(127*model.fc.true_weight()[1].detach().cpu().numpy().ravel()).astype(int)

    model.load_state_dict(torch.load(model_path+str(i+1)+'.ckpt'))
    second0=numpy.round(127*model.conv0.true_weight().detach().cpu().numpy().ravel()).astype(int)
    second1=model.conv1.true_weight().detach().cpu().numpy().ravel()
    ## 1-5为二值化层
    second1=model.conv1.true_weight().detach().cpu().numpy().ravel()
    second2=model.conv2.true_weight().detach().cpu().numpy().ravel()
    second3=model.conv3.true_weight().detach().cpu().numpy().ravel()
    second4=model.conv4.true_weight().detach().cpu().numpy().ravel()
    second5=model.conv5.true_weight().detach().cpu().numpy().ravel()
    second_fc_weight=numpy.round(127*model.fc.true_weight()[0].detach().cpu().numpy().ravel()).astype(int)
    second_fc_bias=numpy.round(127*model.fc.true_weight()[1].detach().cpu().numpy().ravel()).astype(int)
    ####################################################################################
    ## 把两个版本的差量写到文件中

    out_path='./part_diff/'+str(i)+'.out'
    ##清理文件
    with open(out_path,'w') as binfile:
        print('clean the ',out_path)
    ##写入
    with open(out_path,'ab') as binfile:
        #################
        for x,y in zip(first0,second0):
            t=y-x
            if t<0:
                t=t+256
            binfile.write(struct.pack('B',t))
        #################
        xnor_diff=(first1*second1)
        xnor_diff=bi_list_To_uint8_list(xnor_diff)
        for t in xnor_diff:
            binfile.write(struct.pack('B',t))
        #################
        xnor_diff=(first2*second2)
        xnor_diff=bi_list_To_uint8_list(xnor_diff)
        for t in xnor_diff:
            binfile.write(struct.pack('B',t))
        #################
        xnor_diff=(first3*second3)
        xnor_diff=bi_list_To_uint8_list(xnor_diff)
        for t in xnor_diff:
            binfile.write(struct.pack('B',t))
        #################
        xnor_diff=(first4*second4)
        xnor_diff=bi_list_To_uint8_list(xnor_diff)
        for t in xnor_diff:
            binfile.write(struct.pack('B',t))
        #################
        xnor_diff=(first5*second5)
        xnor_diff=bi_list_To_uint8_list(xnor_diff)
        for t in xnor_diff:
            binfile.write(struct.pack('B',t))
        #################
        for x,y in zip(first_fc_weight,second_fc_weight):
            t=y-x
            if t<0:
                t=t+256
            binfile.write(struct.pack('B',t))
        #################
        for x,y in zip(first_fc_bias,second_fc_bias):
            t=y-x
            if t<0:
                t=t+256
            binfile.write(struct.pack('B',t))

  0%|          | 1/753 [00:02<30:51,  2.46s/it]


KeyboardInterrupt: 

In [3]:
from tqdm import tqdm
import struct
import os
import time

def file_clean(path):
    with open(path,'w') as file:
        pass
    print('clean the ',path)
    return

model_path='./out/allnormal89.33/'

model = my_model.VGG_SMALL_1W1A_normal().cuda()

for i in tqdm(range(999)):
#for i in (range(999)):
    model.load_state_dict(torch.load(model_path+str(i)+'.ckpt'))
    first0=numpy.round(127*model.conv0.true_weight().detach().cpu().numpy().ravel()).astype(int)
    first1=numpy.round(127*model.conv1.true_weight().detach().cpu().numpy().ravel()).astype(int)
    first2=numpy.round(127*model.conv2.true_weight().detach().cpu().numpy().ravel()).astype(int)
    first3=numpy.round(127*model.conv3.true_weight().detach().cpu().numpy().ravel()).astype(int)
    first4=numpy.round(127*model.conv4.true_weight().detach().cpu().numpy().ravel()).astype(int)
    first5=numpy.round(127*model.conv5.true_weight().detach().cpu().numpy().ravel()).astype(int)
    first_fc_weight=numpy.round(127*model.fc.true_weight()[0].detach().cpu().numpy().ravel()).astype(int)
    first_fc_bias=numpy.round(127*model.fc.true_weight()[1].detach().cpu().numpy().ravel()).astype(int)

    model.load_state_dict(torch.load(model_path+str(i+1)+'.ckpt'))
    second0=numpy.round(127*model.conv0.true_weight().detach().cpu().numpy().ravel()).astype(int)
    second1=numpy.round(127*model.conv1.true_weight().detach().cpu().numpy().ravel()).astype(int)
    second2=numpy.round(127*model.conv2.true_weight().detach().cpu().numpy().ravel()).astype(int)
    second3=numpy.round(127*model.conv3.true_weight().detach().cpu().numpy().ravel()).astype(int)
    second4=numpy.round(127*model.conv4.true_weight().detach().cpu().numpy().ravel()).astype(int)
    second5=numpy.round(127*model.conv5.true_weight().detach().cpu().numpy().ravel()).astype(int)
    second_fc_weight=numpy.round(127*model.fc.true_weight()[0].detach().cpu().numpy().ravel()).astype(int)
    second_fc_bias=numpy.round(127*model.fc.true_weight()[0].detach().cpu().numpy().ravel()).astype(int)


    out_path='./diff_conv/'+str(i)+'.out'
    file_clean(out_path)

    with open(out_path,'ab') as binfile:
####################################################################################
####################################################################################
# 两个[-128,+127]的数的差值范围为[-255,+255]，远远超过了int8所能表示的范围
# 所以不能直接对两个做差得到差量
# 将[-128,+127]变成一个有向循环的区间，差量的取值范围为[0,255]
# 将差量表示成为下一个版本减上一个版本的值，若值大于0，则直接存储，若值小于0，则用256+结果的值存储（类似反码）
# 在解压时，当前版本加上差量，若结果在[-128,+127]之间，则结束
# 若结果>+127，则将结果减去256（即将+128连接到-128），则可得到正确结果
# 
# 例如:+100->-100,差量=-200(小于0)，则保存-200+256=56
# 恢复时，下一版本=+100+56=156>127，则减去256=156-256=100
# 
####################################################################################
####################################################################################
        for x,y in zip(first0,second0):
            t=y-x
            if t<0:
                t=t+256
            binfile.write(struct.pack('B',t))

        for x,y in zip(first1,second1):
            t=y-x
            if t<0:
                t=t+256
            binfile.write(struct.pack('B',t))

        for x,y in zip(first2,second2):
            t=y-x
            if t<0:
                t=t+256
            binfile.write(struct.pack('B',t))

        for x,y in zip(first3,second3):
            t=y-x
            if t<0:
                t=t+256
            binfile.write(struct.pack('B',t))

        for x,y in zip(first4,second4):
            t=y-x
            if t<0:
                t=t+256
            binfile.write(struct.pack('B',t))

        for x,y in zip(first5,second5):
            t=y-x
            if t<0:
                t=t+256
            binfile.write(struct.pack('B',t))

        for x,y in zip(first_fc_weight,second_fc_weight):
            t=y-x
            if t<0:
                t=t+256
            binfile.write(struct.pack('B',t))

        for x,y in zip(first_fc_bias,second_fc_bias):
            t=y-x
            if t<0:
                t=t+256
            binfile.write(struct.pack('B',t))
            
    print('success write the ',out_path)

  0%|          | 1/999 [00:03<55:22,  3.33s/it]success write the  ./diff_conv/0.out
clean the  ./diff_conv/1.out
  0%|          | 2/999 [00:06<54:40,  3.29s/it]success write the  ./diff_conv/1.out
clean the  ./diff_conv/2.out
  0%|          | 3/999 [00:09<54:44,  3.30s/it]success write the  ./diff_conv/2.out
clean the  ./diff_conv/3.out
  0%|          | 4/999 [00:13<55:05,  3.32s/it]success write the  ./diff_conv/3.out
clean the  ./diff_conv/4.out
  0%|          | 4/999 [00:15<1:03:54,  3.85s/it]


KeyboardInterrupt: 

In [52]:
# 即total_paras参数不包含bn层的running_mean 和 running_var
# 但是包括bn.weight和bn.bias
# 因为这些参数是对输入数据的统计，不是学习得到的
total_num = sum(model.state_dict()[i].reshape(-1).shape[0] for i in model.state_dict())
total_params = sum(p.numel() for p in model.parameters())
print(total_num)
print(total_params)

4663696
4660106
